In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import catboost
from catboost import CatBoostClassifier , Pool
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
# import optuna
# from optuna import Trial
# from optuna.samplers import TPESampler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')
import random

# 랜덤 시드를 고정
random.seed(42)
np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/train (1).csv')
test = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/test (1).csv')
submission = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/submission.csv')

In [ ]:
train

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [ ]:
drop_cols = []
for i in range(len(train.columns)):
    if len(train.iloc[:, i].unique()) == 1:
        drop_cols.append(train.columns[i])
train = train.drop(drop_cols, axis=1)

In [ ]:
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [ ]:
drop_cols = []
for column in train.columns:
  if (train[column].notnull().sum() // 5) < train[
      column].isnull().sum():
      drop_cols.append(column)
train[drop_cols] = train[drop_cols].fillna(0)
train = train
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [ ]:
real_train = train.T.drop_duplicates().T
common_columns = real_train.columns.intersection(train.columns)
real_train = real_train.astype(train[common_columns].dtypes)
real_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 125 entries, Equipment_Dam to target
dtypes: float64(66), int64(48), object(11)
memory usage: 38.6+ MB


In [ ]:
train = real_train

In [ ]:
test = test[train.columns]

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'CURE END POSITION' in col]

# 새로운 'CURE END POSITION' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE END POSITION' in col), None)

# 해당 위치에 새로운 'CURE END POSITION' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE END POSITION', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE END POSITION'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE END POSITION' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'HEAD Standby Position' in col]


# 새로운 'HEAD Standby Position' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'HEAD Standby Position' in col), None)

# 해당 위치에 새로운 'HEAD Standby Position' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'HEAD Standby Position', train[cure_end_position_cols].sum(axis=1))
else:
    train['HEAD Standby Position'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'HEAD Standby Position' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'Head Clean Position'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'Head Clean Position' in col]

# 새로운 'Head Clean Position' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'Head Clean Position' in col), None)

# 해당 위치에 새로운 'Head Clean Position' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'Head Clean Position', train[cure_end_position_cols].sum(axis=1))
else:
    train['Head Clean Position'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'Head Clean Position' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in train.columns if 'CURE START POSITION' in col]

# 새로운 'CURE START POSITION' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE START POSITION' in col), None)

# 해당 위치에 새로운 'CURE START POSITION' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE START POSITION', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE START POSITION'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE START POSITION' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in train.columns if 'CURE SPEED Collect' in col]


# 새로운 'CURE SPEED Collect' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE SPEED Collect' in col), None)

# 해당 위치에 새로운 'CURE SPEED Collect' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE SPEED Collect', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE SPEED Collect'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE SPEED Collect' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION,CURE SPEED Collect,CURE START POSITION,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,...,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,425.5,148,2050,16,14.9,8.4,14.7,...,428.0,243.7,91.8,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,425.5,120,2050,10,21.3,4.9,21.3,...,243.7,243.7,270.0,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1375.5,135,1300,16,14.7,8.5,14.7,...,1324.2,243.5,91.8,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1375.5,120,1300,10,21.3,8.4,21.3,...,243.5,243.5,270.0,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,424.5,120,2050,10,9.7,4.9,9.6,...,243.7,243.7,270.0,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,425.5,120,2050,10,17.0,5.0,17.0,...,243.7,243.7,270.0,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1375.5,148,1300,16,14.9,8.5,14.7,...,1324.2,243.5,91.8,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,425.5,150,2050,16,14.2,8.2,14.3,...,428.0,243.7,91.8,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1375.5,120,1300,10,9.7,4.9,9.7,...,243.5,243.5,270.0,85,20.100,13.0,117,1,0,Normal


# test 파일

In [ ]:
drop_cols = []
for i in range(len(test.columns)):
    if len(test.iloc[:,i].unique()) == 1:
        drop_cols.append(test.columns[i])
test = test.drop(drop_cols, axis=1)

In [ ]:
# Drop columns with more than half of the values missing
drop_cols = []
for column in test.columns:
    if (test[column].notnull().sum() // 5) < test[
        column
    ].isnull().sum():
        drop_cols.append(column)

test[drop_cols] = test[drop_cols].fillna(0)

test = test

test

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,...,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2
0,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,280,10,17.0,...,119,50,91.8,270.0,85,19.8,13.0,195,1,0
1,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,280,16,14.2,...,-10,119,50.0,91.8,50,85.0,19.8,14,256,1
2,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1030,10,9.7,...,119,50,91.8,270.0,85,19.7,1.0,98,1,0
3,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,280,10,21.3,...,-10,119,50.0,91.8,50,85.0,20.0,14,0,1
4,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1030,16,13.2,...,-10,119,50.0,91.8,50,85.0,19.8,1,215,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,280,10,21.3,...,-10,119,50.0,91.8,50,85.0,19.5,14,131,1
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,280,16,13.2,...,-10,119,50.0,91.8,50,85.0,19.8,12,279,1
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1030,16,13.2,...,-10,119,50.0,91.8,50,85.0,20.5,4,66,1
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1030,10,9.7,...,119,50,91.8,270.0,85,18.9,1.0,117,1,0


In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'CURE END POSITION' in col]

# 새로운 'CURE END POSITION' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE END POSITION' in col), None)

# 해당 위치에 새로운 'CURE END POSITION' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE END POSITION', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE END POSITION'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE END POSITION' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'HEAD Standby Position' in col]


# 새로운 'HEAD Standby Position' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'HEAD Standby Position' in col), None)

# 해당 위치에 새로운 'HEAD Standby Position' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'HEAD Standby Position', test[cure_end_position_cols].sum(axis=1))
else:
    test['HEAD Standby Position'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'HEAD Standby Position' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'Head Clean Position'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'Head Clean Position' in col]

# 새로운 'Head Clean Position' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'Head Clean Position' in col), None)

# 해당 위치에 새로운 'Head Clean Position' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'Head Clean Position', test[cure_end_position_cols].sum(axis=1))
else:
    test['Head Clean Position'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'Head Clean Position' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in test.columns if 'CURE START POSITION' in col]

# 새로운 'CURE START POSITION' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE START POSITION' in col), None)

# 해당 위치에 새로운 'CURE START POSITION' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE START POSITION', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE START POSITION'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE START POSITION' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in test.columns if 'CURE SPEED Collect' in col]


# 새로운 'CURE SPEED Collect' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE SPEED Collect' in col), None)

# 해당 위치에 새로운 'CURE SPEED Collect' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE SPEED Collect', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE SPEED Collect'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE SPEED Collect' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
test

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION,CURE SPEED Collect,CURE START POSITION,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,...,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2
0,Dam dispenser #2,AJX75334501,3J1XF767-1,1375.5,120,1300,10,17.0,4.9,17.0,...,1324.2,243.5,243.5,270.0,85,19.8,13.0,195,1,0
1,Dam dispenser #2,AJX75334501,4B1XD472-2,1375.5,120,1300,16,14.2,8.3,14.2,...,1324.2,1324.2,243.5,91.8,50,85.0,19.8,14,256,1
2,Dam dispenser #1,AJX75334501,3H1XE355-1,425.5,120,2050,10,9.7,4.9,9.7,...,428.0,243.7,243.7,270.0,85,19.7,1.0,98,1,0
3,Dam dispenser #2,AJX75334501,3L1XA128-1,1375.5,120,1300,10,21.3,10.6,21.3,...,1324.2,1324.2,243.5,91.8,50,85.0,20.0,14,0,1
4,Dam dispenser #1,AJX75334501,4A1XA639-1,425.5,120,2050,16,13.2,7.5,13.2,...,427.9,428.0,243.7,91.8,50,85.0,19.8,1,215,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,1375.5,120,1300,10,21.3,10.6,21.3,...,1324.2,1324.2,243.5,91.8,50,85.0,19.5,14,131,1
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,1375.5,120,1300,16,13.2,7.6,13.2,...,1324.2,1324.2,243.5,91.8,50,85.0,19.8,12,279,1
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,425.5,120,2050,16,13.2,6.6,13.2,...,427.9,428.0,243.7,91.8,50,85.0,20.5,4,66,1
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,424.5,121,2050,10,9.7,3.9,9.7,...,428.0,243.7,243.7,270.0,85,18.9,1.0,117,1,0


# 피쳐 엔지니어링

In [ ]:
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION,CURE SPEED Collect,CURE START POSITION,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,...,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,425.5,148,2050,16,14.9,8.4,14.7,...,428.0,243.7,91.8,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,425.5,120,2050,10,21.3,4.9,21.3,...,243.7,243.7,270.0,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1375.5,135,1300,16,14.7,8.5,14.7,...,1324.2,243.5,91.8,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1375.5,120,1300,10,21.3,8.4,21.3,...,243.5,243.5,270.0,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,424.5,120,2050,10,9.7,4.9,9.6,...,243.7,243.7,270.0,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,425.5,120,2050,10,17.0,5.0,17.0,...,243.7,243.7,270.0,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1375.5,148,1300,16,14.9,8.5,14.7,...,1324.2,243.5,91.8,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,425.5,150,2050,16,14.2,8.2,14.3,...,428.0,243.7,91.8,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1375.5,120,1300,10,9.7,4.9,9.7,...,243.5,243.5,270.0,85,20.100,13.0,117,1,0,Normal


In [ ]:
test

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION,CURE SPEED Collect,CURE START POSITION,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,...,HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2,HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2
0,Dam dispenser #2,AJX75334501,3J1XF767-1,1375.5,120,1300,10,17.0,4.9,17.0,...,1324.2,243.5,243.5,270.0,85,19.8,13.0,195,1,0
1,Dam dispenser #2,AJX75334501,4B1XD472-2,1375.5,120,1300,16,14.2,8.3,14.2,...,1324.2,1324.2,243.5,91.8,50,85.0,19.8,14,256,1
2,Dam dispenser #1,AJX75334501,3H1XE355-1,425.5,120,2050,10,9.7,4.9,9.7,...,428.0,243.7,243.7,270.0,85,19.7,1.0,98,1,0
3,Dam dispenser #2,AJX75334501,3L1XA128-1,1375.5,120,1300,10,21.3,10.6,21.3,...,1324.2,1324.2,243.5,91.8,50,85.0,20.0,14,0,1
4,Dam dispenser #1,AJX75334501,4A1XA639-1,425.5,120,2050,16,13.2,7.5,13.2,...,427.9,428.0,243.7,91.8,50,85.0,19.8,1,215,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,Dam dispenser #2,AJX75334501,3K1XB597-1,1375.5,120,1300,10,21.3,10.6,21.3,...,1324.2,1324.2,243.5,91.8,50,85.0,19.5,14,131,1
17357,Dam dispenser #2,AJX75334501,4A1XB974-1,1375.5,120,1300,16,13.2,7.6,13.2,...,1324.2,1324.2,243.5,91.8,50,85.0,19.8,12,279,1
17358,Dam dispenser #1,AJX75334501,3L1XA998-1,425.5,120,2050,16,13.2,6.6,13.2,...,427.9,428.0,243.7,91.8,50,85.0,20.5,4,66,1
17359,Dam dispenser #1,AJX75334501,3F1XC376-1,424.5,121,2050,10,9.7,3.9,9.7,...,428.0,243.7,243.7,270.0,85,18.9,1.0,117,1,0


In [ ]:
# '-'를 기준으로 앞부분과 뒷부분을 분리
train['workorder'] = train['Workorder_Dam'].apply(lambda x: x.split('-')[0])
train['workorder_num'] = train['Workorder_Dam'].apply(lambda x: x.split('-')[-1])

test['workorder'] = test['Workorder_Dam'].apply(lambda x: x.split('-')[0])
test['workorder_num'] = test['Workorder_Dam'].apply(lambda x: x.split('-')[-1])

In [ ]:
train['1st Pressure'] = train['1st Pressure 1st Pressure Unit Time_AutoClave'] / train['1st Pressure Collect Result_AutoClave']
train['2nd Pressure'] = train['2nd Pressure Unit Time_AutoClave'] / train['2nd Pressure Collect Result_AutoClave']
train['3rd Pressure'] = train['3rd Pressure Unit Time_AutoClave'] / train['3rd Pressure Collect Result_AutoClave']
train['Chamber Temp AutoClave'] = train['Chamber Temp. Unit Time_AutoClave'] / train['Chamber Temp. Collect Result_AutoClave']

test['1st Pressure'] = test['1st Pressure 1st Pressure Unit Time_AutoClave'] / test['1st Pressure Collect Result_AutoClave']
test['2nd Pressure'] = test['2nd Pressure Unit Time_AutoClave'] / test['2nd Pressure Collect Result_AutoClave']
test['3rd Pressure'] = test['3rd Pressure Unit Time_AutoClave'] / test['3rd Pressure Collect Result_AutoClave']
test['Chamber Temp AutoClave'] = test['Chamber Temp. Unit Time_AutoClave'] / test['Chamber Temp. Collect Result_AutoClave']

In [ ]:
train['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
train['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
train['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

test['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
test['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
test['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

In [ ]:
train['Total Dispense Volume Collect Result_Fill1'] = train['Dispense Volume(Stage1) Collect Result_Fill1'] + train['Dispense Volume(Stage2) Collect Result_Fill1'] + train['Dispense Volume(Stage3) Collect Result_Fill1']

test['Total Dispense Volume Collect Result_Fill1'] = test['Dispense Volume(Stage1) Collect Result_Fill1'] + test['Dispense Volume(Stage2) Collect Result_Fill1'] + test['Dispense Volume(Stage3) Collect Result_Fill1']

In [ ]:
train['Total Process Time'] = train['Machine Tact time Collect Result_Dam'] + train['Machine Tact time Collect Result_Fill1'] + train['Machine Tact time Collect Result_Fill2']

test['Total Process Time'] = test['Machine Tact time Collect Result_Dam'] + test['Machine Tact time Collect Result_Fill1'] + test['Machine Tact time Collect Result_Fill2']

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage1 Circle Result'] = train['Stage1 Circle2 Distance Speed Collect Result_Dam'] + train['Stage1 Circle1 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage1 Circle Result'] = test['Stage1 Circle2 Distance Speed Collect Result_Dam'] + test['Stage1 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
# train 데이터셋에서 두 열의 값을 더하여 새로운 열을 생성
train['Stage2 Circle Result'] = train['Stage2 Circle2 Distance Speed Collect Result_Dam'] + train['Stage2 Circle1 Distance Speed Collect Result_Dam']

# test 데이터셋에서 동일한 작업 수행
test['Stage2 Circle Result'] = test['Stage2 Circle2 Distance Speed Collect Result_Dam'] + test['Stage2 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
# train 데이터셋에서 두 열의 값을 더하여 새로운 열을 생성
train['Stage3 Circle Result'] = train['Stage3 Circle2 Distance Speed Collect Result_Dam'] + train['Stage3 Circle1 Distance Speed Collect Result_Dam']

# test 데이터셋에서 동일한 작업 수행
test['Stage3 Circle Result'] = test['Stage3 Circle2 Distance Speed Collect Result_Dam'] + test['Stage3 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
train['Total Production'] = train['Production Qty Collect Result_Dam'] + train['Production Qty Collect Result_Fill1'] + train['Production Qty Collect Result_Fill2']

test['Total Production'] = test['Production Qty Collect Result_Dam'] + test['Production Qty Collect Result_Fill1'] + test['Production Qty Collect Result_Fill2']

In [ ]:
train['Workorder_Dam_Middle'] = train['Workorder_Dam'].str.slice(3, 5)

test['Workorder_Dam_Middle'] = test['Workorder_Dam'].str.slice(3, 5)

In [ ]:
train['Workorder_Dam_First'] = train['Workorder_Dam'].str.slice(0, 2)

test['Workorder_Dam_First'] = test['Workorder_Dam'].str.slice(0, 2)

In [ ]:
# # OK 값을 가지는 컬럼을 저장할 리스트
# ok_columns = [col for col in train.columns if 'OK' in train[col].unique()]

# # 결과 출력
# print(ok_columns)

In [ ]:
# 변경할 열들의 리스트
columns_to_replace = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 각 열의 문자 값을 0으로 변경
train[columns_to_replace] = train[columns_to_replace].applymap(lambda x: 0 if isinstance(x, str) else x)
test[columns_to_replace] = test[columns_to_replace].applymap(lambda x: 0 if isinstance(x, str) else x)

In [ ]:
train.drop(['workorder'], axis=1, inplace=True)

test.drop(['workorder'], axis=1, inplace=True)

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage1 Line Result'] = train['Stage1 Line2 Distance Speed Collect Result_Dam'] + train['Stage1 Line1 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage1 Line Result'] = test['Stage1 Line2 Distance Speed Collect Result_Dam'] + test['Stage1 Line1 Distance Speed Collect Result_Dam']

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage2 Line Result'] = train['Stage2 Line2 Distance Speed Collect Result_Dam'] + train['Stage2 Line3 Distance Speed Collect Result_Dam'] + train['Stage2 Line4 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage2 Line Result'] = test['Stage2 Line2 Distance Speed Collect Result_Dam'] + test['Stage2 Line3 Distance Speed Collect Result_Dam'] + train['Stage2 Line4 Distance Speed Collect Result_Dam']

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage3 Line Result'] = train['Stage3 Line2 Distance Speed Collect Result_Dam'] + train['Stage3 Line1 Distance Speed Collect Result_Dam'] + train['Stage3 Line4 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage3 Line Result'] = test['Stage3 Line2 Distance Speed Collect Result_Dam'] + test['Stage3 Line1 Distance Speed Collect Result_Dam'] + train['Stage3 Line4 Distance Speed Collect Result_Dam']

In [ ]:
train['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
train['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
train['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Dam'] = train['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

test['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']
test['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']
test['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Dam'] = test['DISCHARGED SPEED OF RESIN Collect Result_Dam'] * test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']

In [ ]:
# 기존 코드 유지
categorical_feat = train.columns.tolist()
len(categorical_feat)

categorical_feat.remove('target')

train[categorical_feat] = train[categorical_feat].astype('str')
test[categorical_feat] = test[categorical_feat].astype('str')

# cat_cols를 인덱스 대신 열 이름으로 설정
cat_cols = categorical_feat

In [ ]:
train[categorical_feat] = train[categorical_feat].astype('str')
test[categorical_feat] = test[categorical_feat].astype('str')

In [ ]:
# cat_cols를 인덱스 대신 열 이름으로 설정
cat_cols = categorical_feat

In [ ]:
train['target'] = train['target'].replace({'Normal': 0, 'AbNormal': 1})

In [ ]:
stratify_columns = train[["target", "Workorder_Dam_Middle"]]

In [ ]:
# 데이터를 stratified split
x_trn, x_val = train_test_split(
    train, test_size=0.2, random_state=42, stratify=stratify_columns)

In [ ]:
y_trn = x_trn['target']
x_trn = x_trn.drop(['target'], axis=1)

y_val = x_val['target']
x_val = x_val.drop(['target'], axis=1)

In [ ]:
train2 = train.copy()
test2 = test.copy()

# 앙상블 data 준비

### train

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/train (1).csv')
test = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/test (1).csv')
submission = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/submission.csv')

In [ ]:
drop_cols = []
for i in range(len(train.columns)):
    if len(train.iloc[:, i].unique()) == 1:
        drop_cols.append(train.columns[i])
train = train.drop(drop_cols, axis=1)

In [ ]:
drop_cols = []
for column in train.columns:
  if (train[column].notnull().sum() // 5) < train[
      column].isnull().sum():
      drop_cols.append(column)
train[drop_cols] = train[drop_cols].fillna(0)
train = train
train

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,90,16,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,-90,10,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,90,16,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,-90,16,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,90,10,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [ ]:
real_train = train.T.drop_duplicates().T
common_columns = real_train.columns.intersection(train.columns)
real_train = real_train.astype(train[common_columns].dtypes)
real_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 125 entries, Equipment_Dam to target
dtypes: float64(66), int64(48), object(11)
memory usage: 38.6+ MB


In [ ]:
train = real_train

In [ ]:
test = test[train.columns]

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'CURE END POSITION' in col]

# 새로운 'CURE END POSITION' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE END POSITION' in col), None)

# 해당 위치에 새로운 'CURE END POSITION' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE END POSITION', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE END POSITION'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE END POSITION' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'HEAD Standby Position' in col]


# 새로운 'HEAD Standby Position' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'HEAD Standby Position' in col), None)

# 해당 위치에 새로운 'HEAD Standby Position' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'HEAD Standby Position', train[cure_end_position_cols].sum(axis=1))
else:
    train['HEAD Standby Position'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'HEAD Standby Position' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'Head Clean Position'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in train.columns if 'Head Clean Position' in col]

# 새로운 'Head Clean Position' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'Head Clean Position' in col), None)

# 해당 위치에 새로운 'Head Clean Position' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'Head Clean Position', train[cure_end_position_cols].sum(axis=1))
else:
    train['Head Clean Position'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'Head Clean Position' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in train.columns if 'CURE START POSITION' in col]

# 새로운 'CURE START POSITION' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE START POSITION' in col), None)

# 해당 위치에 새로운 'CURE START POSITION' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE START POSITION', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE START POSITION'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE START POSITION' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in train.columns if 'CURE SPEED Collect' in col]


# 새로운 'CURE SPEED Collect' 열의 위치 찾기
cols = list(train.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE SPEED Collect' in col), None)

# 해당 위치에 새로운 'CURE SPEED Collect' 열 추가
if cure_end_position_idx is not None:
    train.insert(cure_end_position_idx, 'CURE SPEED Collect', train[cure_end_position_cols].sum(axis=1))
else:
    train['CURE SPEED Collect'] = train[cure_end_position_cols].sum(axis=1)

# 기존 'CURE SPEED Collect' 관련 열들 삭제
train = train.drop(cure_end_position_cols, axis=1)

### test

In [ ]:
drop_cols = []
for i in range(len(test.columns)):
    if len(test.iloc[:,i].unique()) == 1:
        drop_cols.append(test.columns[i])
test = test.drop(drop_cols, axis=1)

In [ ]:
# Drop columns with more than half of the values missing
drop_cols = []
for column in test.columns:
    if (test[column].notnull().sum() // 5) < test[
        column
    ].isnull().sum():
        drop_cols.append(column)

test[drop_cols] = test[drop_cols].fillna(0)

test = test

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'CURE END POSITION' in col]

# 새로운 'CURE END POSITION' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE END POSITION' in col), None)

# 해당 위치에 새로운 'CURE END POSITION' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE END POSITION', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE END POSITION'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE END POSITION' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'CURE END POSITION'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'HEAD Standby Position' in col]


# 새로운 'HEAD Standby Position' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'HEAD Standby Position' in col), None)

# 해당 위치에 새로운 'HEAD Standby Position' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'HEAD Standby Position', test[cure_end_position_cols].sum(axis=1))
else:
    test['HEAD Standby Position'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'HEAD Standby Position' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# 'Head Clean Position'이라는 문자열이 포함된 열들의 목록 가져오기
cure_end_position_cols = [col for col in test.columns if 'Head Clean Position' in col]

# 새로운 'Head Clean Position' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'Head Clean Position' in col), None)

# 해당 위치에 새로운 'Head Clean Position' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'Head Clean Position', test[cure_end_position_cols].sum(axis=1))
else:
    test['Head Clean Position'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'Head Clean Position' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in test.columns if 'CURE START POSITION' in col]

# 새로운 'CURE START POSITION' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE START POSITION' in col), None)

# 해당 위치에 새로운 'CURE START POSITION' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE START POSITION', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE START POSITION'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE START POSITION' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
cure_end_position_cols = [col for col in test.columns if 'CURE SPEED Collect' in col]


# 새로운 'CURE SPEED Collect' 열의 위치 찾기
cols = list(test.columns)
cure_end_position_idx = next((i for i, col in enumerate(cols) if 'CURE SPEED Collect' in col), None)

# 해당 위치에 새로운 'CURE SPEED Collect' 열 추가
if cure_end_position_idx is not None:
    test.insert(cure_end_position_idx, 'CURE SPEED Collect', test[cure_end_position_cols].sum(axis=1))
else:
    test['CURE SPEED Collect'] = test[cure_end_position_cols].sum(axis=1)

# 기존 'CURE SPEED Collect' 관련 열들 삭제
test = test.drop(cure_end_position_cols, axis=1)

In [ ]:
# '-'를 기준으로 앞부분과 뒷부분을 분리
train['workorder'] = train['Workorder_Dam'].apply(lambda x: x.split('-')[0])
train['workorder_num'] = train['Workorder_Dam'].apply(lambda x: x.split('-')[-1])

test['workorder'] = test['Workorder_Dam'].apply(lambda x: x.split('-')[0])
test['workorder_num'] = test['Workorder_Dam'].apply(lambda x: x.split('-')[-1])

In [ ]:
train['1st Pressure'] = train['1st Pressure 1st Pressure Unit Time_AutoClave'] / train['1st Pressure Collect Result_AutoClave']
train['2nd Pressure'] = train['2nd Pressure Unit Time_AutoClave'] / train['2nd Pressure Collect Result_AutoClave']
train['3rd Pressure'] = train['3rd Pressure Unit Time_AutoClave'] / train['3rd Pressure Collect Result_AutoClave']
train['Chamber Temp AutoClave'] = train['Chamber Temp. Unit Time_AutoClave'] / train['Chamber Temp. Collect Result_AutoClave']

test['1st Pressure'] = test['1st Pressure 1st Pressure Unit Time_AutoClave'] / test['1st Pressure Collect Result_AutoClave']
test['2nd Pressure'] = test['2nd Pressure Unit Time_AutoClave'] / test['2nd Pressure Collect Result_AutoClave']
test['3rd Pressure'] = test['3rd Pressure Unit Time_AutoClave'] / test['3rd Pressure Collect Result_AutoClave']
test['Chamber Temp AutoClave'] = test['Chamber Temp. Unit Time_AutoClave'] / test['Chamber Temp. Collect Result_AutoClave']

In [ ]:
train['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
train['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
train['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Fill1'] = train['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * train['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

test['DISCHARGED DISTANCE(Stage1) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']
test['DISCHARGED DISTANCE(Stage2) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']
test['DISCHARGED DISTANCE(Stage3) OF RESIN Collect Result_Fill1'] = test['DISCHARGED SPEED OF RESIN Collect Result_Fill1'] * test['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']

In [ ]:
train['Total Dispense Volume Collect Result_Fill1'] = train['Dispense Volume(Stage1) Collect Result_Fill1'] + train['Dispense Volume(Stage2) Collect Result_Fill1'] + train['Dispense Volume(Stage3) Collect Result_Fill1']

test['Total Dispense Volume Collect Result_Fill1'] = test['Dispense Volume(Stage1) Collect Result_Fill1'] + test['Dispense Volume(Stage2) Collect Result_Fill1'] + test['Dispense Volume(Stage3) Collect Result_Fill1']

In [ ]:
train['Total Process Time'] = train['Machine Tact time Collect Result_Dam'] + train['Machine Tact time Collect Result_Fill1'] + train['Machine Tact time Collect Result_Fill2']

test['Total Process Time'] = test['Machine Tact time Collect Result_Dam'] + test['Machine Tact time Collect Result_Fill1'] + test['Machine Tact time Collect Result_Fill2']

In [ ]:
# 두 열의 값을 더해서 새로운 열을 생성
train['Stage1 Circle Result'] = train['Stage1 Circle2 Distance Speed Collect Result_Dam'] + train['Stage1 Circle1 Distance Speed Collect Result_Dam']

# 동일한 작업을 test 데이터셋에 적용
test['Stage1 Circle Result'] = test['Stage1 Circle2 Distance Speed Collect Result_Dam'] + test['Stage1 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
# train 데이터셋에서 두 열의 값을 더하여 새로운 열을 생성
train['Stage2 Circle Result'] = train['Stage2 Circle2 Distance Speed Collect Result_Dam'] + train['Stage2 Circle1 Distance Speed Collect Result_Dam']

# test 데이터셋에서 동일한 작업 수행
test['Stage2 Circle Result'] = test['Stage2 Circle2 Distance Speed Collect Result_Dam'] + test['Stage2 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
# train 데이터셋에서 두 열의 값을 더하여 새로운 열을 생성
train['Stage3 Circle Result'] = train['Stage3 Circle2 Distance Speed Collect Result_Dam'] + train['Stage3 Circle1 Distance Speed Collect Result_Dam']

# test 데이터셋에서 동일한 작업 수행
test['Stage3 Circle Result'] = test['Stage3 Circle2 Distance Speed Collect Result_Dam'] + test['Stage3 Circle1 Distance Speed Collect Result_Dam']

In [ ]:
train['Total Production'] = train['Production Qty Collect Result_Dam'] + train['Production Qty Collect Result_Fill1'] + train['Production Qty Collect Result_Fill2']

test['Total Production'] = test['Production Qty Collect Result_Dam'] + test['Production Qty Collect Result_Fill1'] + test['Production Qty Collect Result_Fill2']

In [ ]:
train['Workorder_Dam_Middle'] = train['Workorder_Dam'].str.slice(3, 5)

test['Workorder_Dam_Middle'] = test['Workorder_Dam'].str.slice(3, 5)

In [ ]:
train['Workorder_Dam_First'] = train['Workorder_Dam'].str.slice(0, 2)

test['Workorder_Dam_First'] = test['Workorder_Dam'].str.slice(0, 2)

In [ ]:
# 변경할 열들의 리스트
columns_to_replace = [
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1',
    'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'
]

# 각 열의 문자 값을 0으로 변경
train[columns_to_replace] = train[columns_to_replace].applymap(lambda x: 0 if isinstance(x, str) else x)
test[columns_to_replace] = test[columns_to_replace].applymap(lambda x: 0 if isinstance(x, str) else x)

In [ ]:
train.drop(['Stage1 Line1 Distance Speed Collect Result_Dam', 'Stage1 Line2 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

test.drop(['Stage1 Line1 Distance Speed Collect Result_Dam', 'Stage1 Line2 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

In [ ]:
train.drop(['Stage2 Line2 Distance Speed Collect Result_Dam', 'Stage2 Line3 Distance Speed Collect Result_Dam', 'Stage2 Line4 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

test.drop(['Stage2 Line2 Distance Speed Collect Result_Dam', 'Stage2 Line3 Distance Speed Collect Result_Dam', 'Stage2 Line4 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

In [ ]:
train.drop(['Stage3 Line1 Distance Speed Collect Result_Dam', 'Stage3 Line2 Distance Speed Collect Result_Dam', 'Stage3 Line4 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

test.drop(['Stage3 Line1 Distance Speed Collect Result_Dam', 'Stage3 Line2 Distance Speed Collect Result_Dam', 'Stage3 Line4 Distance Speed Collect Result_Dam'], axis=1, inplace=True)

In [ ]:
train1 = train
test1 = test

# 앙상블 전처리

In [ ]:
# 기존 코드 유지
categorical_feat = train1.columns.tolist()
len(categorical_feat)

categorical_feat.remove('target')

train1[categorical_feat] = train1[categorical_feat].astype('str')
test1[categorical_feat] = test1[categorical_feat].astype('str')

# cat_cols를 인덱스 대신 열 이름으로 설정
cat_cols_1 = categorical_feat

In [ ]:
train1[categorical_feat] = train1[categorical_feat].astype('str')
test1[categorical_feat] = test1[categorical_feat].astype('str')

In [ ]:
# cat_cols를 인덱스 대신 열 이름으로 설정
cat_cols_1 = categorical_feat

In [ ]:
train1['target'] = train1['target'].replace({'Normal': 0, 'AbNormal': 1})

In [ ]:
stratify_columns = train1[["target", "Workorder_Dam_Middle"]]

In [ ]:
# 데이터를 stratified split
x_trn_1, x_val_1 = train_test_split(
    train1, test_size=0.2, random_state=42, stratify=stratify_columns)

In [ ]:
y_trn_1 = x_trn_1['target']
x_trn_1 = x_trn_1.drop(['target'], axis=1)

y_val_1 = x_val_1['target']
x_val_1 = x_val_1.drop(['target'], axis=1)

# 모델링

In [ ]:
predict = np.zeros((len(x_val), 2))

In [ ]:
cat_model_1 = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='F1',
    iterations=3724,
    depth=5,
    l2_leaf_reg=7.35,
    min_data_in_leaf=7,
    scale_pos_weight=7.97,
    learning_rate= 0.01,
    random_seed=42,
    cat_features=cat_cols_1,
    early_stopping_rounds=1000,
)

In [ ]:
cat_model_1.fit(x_trn_1, y_trn_1, verbose=500)

0:	learn: 0.0406063	total: 131ms	remaining: 8m 8s
500:	learn: 0.2677352	total: 1m 2s	remaining: 6m 43s
1000:	learn: 0.3220595	total: 2m 7s	remaining: 5m 46s
1500:	learn: 0.3850447	total: 3m 18s	remaining: 4m 53s
2000:	learn: 0.4273084	total: 4m 29s	remaining: 3m 52s
2500:	learn: 0.4578434	total: 5m 41s	remaining: 2m 46s
3000:	learn: 0.4863919	total: 6m 53s	remaining: 1m 39s
3500:	learn: 0.5165862	total: 8m 6s	remaining: 31s
3723:	learn: 0.5274067	total: 8m 39s	remaining: 0us


In [ ]:
pred = cat_model_1.predict_proba(x_val_1)[:, 1]
predict[:, 0] = pred

In [ ]:
cat_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='F1',
    iterations=3999,
    depth=5,
    l2_leaf_reg=7.35,
    min_data_in_leaf=7,
    scale_pos_weight=7.97,
    learning_rate= 0.01,
    random_seed=42,
    cat_features=cat_cols,
    early_stopping_rounds=1000,
)

In [ ]:
cat_model.fit(x_trn, y_trn, verbose=500)

0:	learn: 0.0617517	total: 165ms	remaining: 11m
500:	learn: 0.2606866	total: 1m 8s	remaining: 7m 55s
1000:	learn: 0.3145117	total: 2m 19s	remaining: 6m 59s
1500:	learn: 0.3750429	total: 3m 38s	remaining: 6m 3s
2000:	learn: 0.4138600	total: 4m 57s	remaining: 4m 57s
2500:	learn: 0.4539382	total: 6m 17s	remaining: 3m 45s
3000:	learn: 0.4839414	total: 7m 37s	remaining: 2m 32s
3500:	learn: 0.5095966	total: 8m 57s	remaining: 1m 16s
3998:	learn: 0.5319239	total: 10m 18s	remaining: 0us


In [ ]:
pred = cat_model.predict_proba(x_val)[:, 1]
predict[:, 1] = pred

In [ ]:
np.mean(predict, axis = 1) * 100

array([25.95924493, 16.8611425 , 17.36256918, ..., 23.51958114,
       23.05113324, 25.18998858])

In [ ]:
weights = np.array([0.35, 0.65])

# weighted average 계산
weighted_avg = np.sum(predict * weights, axis=1)

print(weighted_avg)

[0.34850556 0.15835064 0.38261807 ... 0.2820954  0.38736116 0.1771976 ]


In [ ]:
result = np.where(weighted_avg >= 0.5, 1, 0)
sum(result)

1065

In [ ]:
f1_val = f1_score(y_val, result, pos_label= 1)
print('F1_val:', f1_val)

F1_val: 0.2785114045618247


#제출파일 생성

In [ ]:
predict = np.zeros((len(test1), 2))

In [ ]:
cat_model_1.fit(train1.drop(['target'], axis=1), train1['target'], verbose=500)

0:	learn: 0.0601375	total: 155ms	remaining: 9m 36s
500:	learn: 0.3199781	total: 1m 18s	remaining: 8m 22s
1000:	learn: 0.3646113	total: 2m 38s	remaining: 7m 9s
1500:	learn: 0.4077689	total: 4m 5s	remaining: 6m 3s
2000:	learn: 0.4361048	total: 5m 34s	remaining: 4m 48s
2500:	learn: 0.4592379	total: 7m 3s	remaining: 3m 26s
3000:	learn: 0.4810806	total: 8m 33s	remaining: 2m 3s
3500:	learn: 0.4983398	total: 10m 3s	remaining: 38.4s
3723:	learn: 0.5086313	total: 10m 43s	remaining: 0us


In [ ]:
cat_model.fit(train2.drop(['target'], axis=1), train2['target'], verbose=500)

0:	learn: 0.1003740	total: 197ms	remaining: 13m 7s
500:	learn: 0.3185677	total: 1m 27s	remaining: 10m 13s
1000:	learn: 0.3612170	total: 2m 56s	remaining: 8m 47s
1500:	learn: 0.4081892	total: 4m 32s	remaining: 7m 34s
2000:	learn: 0.4365974	total: 6m 11s	remaining: 6m 10s
2500:	learn: 0.4592203	total: 7m 50s	remaining: 4m 41s
3000:	learn: 0.4764013	total: 9m 29s	remaining: 3m 9s
3500:	learn: 0.4991259	total: 11m 10s	remaining: 1m 35s
3998:	learn: 0.5168063	total: 12m 49s	remaining: 0us


In [ ]:
pred = cat_model_1.predict_proba(test1)[:, 1]
predict[:, 0] = pred

In [ ]:
pred = cat_model.predict_proba(test2)[:, 1]
predict[:, 1] = pred

In [ ]:
np.mean(predict, axis = 1) * 100

array([34.18171202, 15.5987937 , 38.00289304, ..., 27.84092794,
       39.00887813, 17.2724884 ])

In [ ]:
weights = np.array([0.3, 0.7])

# weighted average 계산
weighted_avg = np.sum(predict * weights, axis=1)

print(weighted_avg)

[0.35073504 0.15913821 0.38348111 ... 0.28332411 0.38645195 0.1786885 ]


In [ ]:
result = np.where(weighted_avg >= 0.5, 1, 0)
sum(result)

1080

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/LG_AIMERS/sample_submission.csv')
submission['target'] = result
submission

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,0
1,0005bbd180064abd99e63f9ed3e1ac80,0
2,000948934c4140d883d670adcb609584,0
3,000a6bfd02874c6296dc7b2e9c5678a7,0
4,0018e78ce91343678716e2ea27a51c95,0
...,...,...
17356,ffea508b59934d689b540f95eb3fa730,0
17357,ffed8923c8a448a98afc641b770be153,0
17358,fff1e73734da40adbe805359b3efb462,0
17359,fff8e38bdd09470baf95f71e92075dec,0


In [ ]:
submission['target'].replace({1: 'AbNormal', 0:'Normal'}, inplace=True)
submission['target'].value_counts()

,count
target,
Normal,16281
AbNormal,1080


In [ ]:
# 첫 번째 조건을 정의합니다
condition1 = (
    (test['Production Qty Collect Result_Dam'] != test['Production Qty Collect Result_Fill1']) |
    (test['Production Qty Collect Result_Fill1'] != test['Production Qty Collect Result_Fill2']) |
    (test['Production Qty Collect Result_Dam'] != test['Production Qty Collect Result_Fill2'])
)

# 두 번째 조건을 정의합니다
condition2 = (
    (test['Receip No Collect Result_Dam'] != test['Receip No Collect Result_Fill1']) |
    (test['Receip No Collect Result_Fill1'] != test['Receip No Collect Result_Fill2']) |
    (test['Receip No Collect Result_Dam'] != test['Receip No Collect Result_Fill2'])
)

# 두 조건을 합칩니다
combined_condition = condition1 | condition2

# test 데이터프레임에서 조건에 해당하는 행의 'target' 열 값을 'AbNormal'로 변경합니다
test.loc[combined_condition, 'target'] = 'AbNormal'

# 조건에 해당하는 행의 인덱스를 가져옵니다
abnormal_indices = test[combined_condition].index

# df_sub 데이터프레임에서 해당 인덱스의 'target' 열 값을 'AbNormal'로 변경합니다
# 'Set ID'를 인덱스로 사용한다고 가정합니다. 만약 다른 열을 사용한다면 이 부분을 수정해야 합니다.
submission.loc[submission.index.isin(abnormal_indices), 'target'] = 'AbNormal'

# 결과 확인
submission['target'].value_counts()

,count
target,
Normal,16281
AbNormal,1080


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,AbNormal
9,002d904240d84b188d410d16383a9c3a,Normal


In [ ]:
from google.colab import files

# submission.csv 파일 다운로드
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>